In [15]:
import numpy as np
import pandas as pd
from os import path
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter
import sklearn
from tensorflow.keras.callbacks import LearningRateScheduler

from sklearn import preprocessing, metrics
from sklearn.metrics import (roc_curve, auc, accuracy_score, precision_score, 
                             recall_score, f1_score, balanced_accuracy_score, 
                             matthews_corrcoef)
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, LabelBinarizer
import shap
from imblearn.over_sampling import RandomOverSampler
import innvestigate

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
np.random.seed(0)

tf.compat.v1.disable_eager_execution()

output_file_name = 'output.txt'
with open(output_file_name, "w") as f: print('--------------------------------------------------------------------------', file = f)

In [16]:
def confusion_metrics (name_model,predictions,true_labels):

    name = name_model
    pred_label = predictions
    y_test_01 = true_labels 

    with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print(name, file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    # voting_acc_01 = Acc
    # voting_pre_01 = Precision
    # weighed_avg_rec_01 = Recall
    # weighed_avg_f1_01 = F1
    # weighed_avg_bacc_01 = BACC
    # weighed_avg_mcc_01 = MCC
    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)

    return Acc, Precision, Recall, F1, BACC, MCC

In [ ]:

req_cols = [
    
    ' Packet Length Std', ' Total Length of Bwd Packets', ' Subflow Bwd Bytes',
    ' Destination Port', ' Packet Length Variance', ' Bwd Packet Length Mean',' Avg Bwd Segment Size',
    'Bwd Packet Length Max', ' Init_Win_bytes_backward','Total Length of Fwd Packets',
    ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' Average Packet Size', ' Packet Length Mean',
    ' Max Packet Length',
    ' Down/Up Ratio', ' Fwd URG Flags', ' Flow IAT Std', 'Subflow Fwd Packets', ' Flow Packets/s', 
    ' URG Flag Count', 'FIN Flag Count', ' Bwd Packets/s', 'Bwd Avg Bulk Rate'
    , ' act_data_pkt_fwd', ' Fwd Packet Length Std', ' Bwd Avg Bytes/Bulk', ' Active Max', 
    ' Flow IAT Max', ' min_seg_size_forward', ' Bwd Packet Length Std', ' Fwd IAT Std', 
    ' Fwd Avg Bulk Rate', ' Fwd Packet Length Mean', ' Fwd Packet Length Max', ' Idle Std', 
    ' CWE Flag Count', 'Fwd IAT Total'
    
    , ' ACK Flag Count', ' Bwd URG Flags', ' Flow IAT Min', ' Flow IAT Mean', 
    ' Total Backward Packets', ' Fwd Avg Packets/Bulk', 'Fwd Avg Bytes/Bulk', 
    ' SYN Flag Count', ' Min Packet Length', ' Fwd Packet Length Min', 'Idle Mean',
      'Fwd PSH Flags', ' Fwd IAT Min'
     
    ,  ' Fwd Header Length', ' RST Flag Count', ' Idle Max', ' PSH Flag Count', 
    ' Bwd Header Length', ' ECE Flag Count', ' Subflow Bwd Packets', 'Active Mean',
      'Flow Bytes/s', ' Bwd IAT Mean', ' Avg Fwd Segment Size', ' Bwd Packet Length Min', 
      ' Active Std', ' Bwd IAT Min', ' Flow Duration', 'Fwd Packets/s', ' Fwd IAT Max', 
      'Bwd IAT Total', ' Idle Min', ' Bwd PSH Flags', ' Bwd Avg Packets/Bulk', ' Total Fwd Packets',
        ' Active Min', ' Bwd IAT Std', ' Fwd IAT Mean', ' Bwd IAT Max'
            
            , ' Label']

# Information gain top 10 features
top10 = [' Average Packet Size',
          ' Packet Length Std', 
          ' Packet Length Variance', 
          ' Packet Length Mean',
            ' Destination Port', 
            ' Subflow Bwd Bytes', 
            ' Total Length of Bwd Packets', 
            ' Avg Bwd Segment Size', 
            ' Bwd Packet Length Mean',  
            'Bwd Packet Length Max', 
            ' Label']

top20 = [
    ' Average Packet Size', 
    ' Packet Length Variance', 
    ' Packet Length Std', 
    ' Packet Length Mean',
    ' Destination Port', 

    ' Total Length of Bwd Packets', 
    ' Subflow Bwd Bytes', 
    ' Avg Bwd Segment Size',
    ' Bwd Packet Length Mean', 
    'Bwd Packet Length Max', 
    
    ' Subflow Fwd Bytes', 
    'Total Length of Fwd Packets',
    ' Init_Win_bytes_backward', 
    'Init_Win_bytes_forward', 
    ' Max Packet Length', 
    
    ' Fwd Packet Length Max',
    ' Flow IAT Max', 
    ' Bwd Packets/s', 
    ' Flow Duration', 
    ' Fwd IAT Max',
    
    ' Label'
]


req_cols = top20

# req_cols = [' Destination Port',' Flow Duration',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Label']
#---------------------------------------------------------------------
#Load Databases from csv file

path_str = '/home/oarreche@ads.iu.edu/HITL/cicids/cicids_db/'
fraction = 1
#---------------------------------------------------------------------
#Load Databases from csv file
print('---------------------------------------------------------------------------------')
print('Loading Databases')
print('---------------------------------------------------------------------------------')
print('')


df0 = pd.read_csv (path_str + 'Wednesday-workingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

df1 = pd.read_csv (path_str + 'Tuesday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df2 = pd.read_csv (path_str +'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df3 = pd.read_csv (path_str +'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df4 = pd.read_csv (path_str +'Monday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df5 = pd.read_csv (path_str +'Friday-WorkingHours-Morning.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df6 = pd.read_csv (path_str +'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df7 = pd.read_csv (path_str +'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

frames = [df0, df1, df2, df3, df4, df5, df6, df7]

df = pd.concat(frames,ignore_index=True)

df = df.sample(frac = 1)

# Eliminate duplicate rows
df = df.drop_duplicates()



# Merged popping unwandted columns descriptive accuracy
##########################################



df.pop(' Packet Length Variance')
df.pop(' Packet Length Std')
df.pop(' Fwd IAT Max')
df.pop(' Flow Duration')
df.pop(' Flow IAT Max')

df.pop(' Destination Port')
df.pop(' Average Packet Size')
df.pop('Init_Win_bytes_forward')
df.pop(' Max Packet Length')
df.pop(' Bwd Packet Length Mean')

df.pop(' Avg Bwd Segment Size')
df.pop('Bwd Packet Length Max')
df.pop(' Bwd Packets/s')
df.pop(' Fwd Packet Length Max')
df.pop(' Packet Length Mean')

# df.pop(' Init_Win_bytes_backward')
# df.pop(' Subflow Bwd Bytes')
# df.pop(' Subflow Fwd Bytes')
# df.pop(' Total Length of Bwd Packets')
# df.pop('Total Length of Fwd Packets')

#---------------------------------------------------------------------
# Normalize database
print('---------------------------------------------------------------------------------')
print('Normalizing database')
print('---------------------------------------------------------------------------------')
print('')

df_max_scaled = df.copy()

y = df_max_scaled[' Label'].replace({'DDoS' :'Dos/Ddos' ,'DoS GoldenEye': 'Dos/Ddos', 'DoS Hulk': 'Dos/Ddos', 'DoS Slowhttptest': 'Dos/Ddos', 'DoS slowloris': 'Dos/Ddos', 'Heartbleed': 'Dos/Ddos','FTP-Patator': 'Brute Force', 'SSH-Patator': 'Brute Force','Web Attack - Brute Force': 'Web Attack', 'Web Attack - Sql Injection': 'Web Attack', 'Web Attack - XSS': 'Web Attack'})

df_max_scaled.pop(' Label')


df_max_scaled
for col in df_max_scaled.columns:
    t = abs(df_max_scaled[col].max())
    df_max_scaled[col] = df_max_scaled[col]/t
df_max_scaled
df = df_max_scaled.assign( Label = y)
#df
df = df.fillna(0)

#---------------------------------------------------------------------
df = df[df['Label'] != 'Brute Force']
df = df[df['Label'] != 'Bot']
df = df[df['Label'] != 'Web Attack']
df = df[df['Label'] != 'Infiltration']


# Separate features and labels 
print('---------------------------------------------------------------------------------')
print('Separating features and labels')
print('---------------------------------------------------------------------------------')
print('')

y = df.pop('Label')
X = df
# summarize class distribution
counter = Counter(y)
print(counter)

df = X.assign( Label = y)

y, label = pd.factorize(y)

# Separate Training and Testing db

# SMOTE

from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto', random_state=42)  
X, y = smote.fit_resample(X, y)

print(pd.Series(y).value_counts())  # Verify balancing
print('---------------------------------------------------------------------------------')
print('Separating Training and Testing db')
print('---------------------------------------------------------------------------------')
print('')

X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.7,random_state=42)
df = X.assign( Label = y)




---------------------------------------------------------------------------------
Loading Databases
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Normalizing database
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Separating features and labels
---------------------------------------------------------------------------------

Counter({'BENIGN': 2050510, 'Dos/Ddos': 320807, 'PortScan': 90819})
0    2050510
1    2050510
2    2050510
Name: count, dtype: int64
---------------------------------------------------------------------------------
Separating Training and Testing db
---------------------------------------------------------------------------------



In [18]:

print('---------------------------------------------------------------------------------')
print('Defining the DNN model')
print('---------------------------------------------------------------------------------')
print('')
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights to handle imbalance
# class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# class_weights = dict(enumerate(class_weights))

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(len(X_train.columns,))))


model.add(tf.keras.layers.Dense(60, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(30, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(15, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(7, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

# model.add(tf.keras.layers.Dense(7,activation='softmax'))

model.add(tf.keras.layers.Dense(4))





optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])




---------------------------------------------------------------------------------
Defining the DNN model
---------------------------------------------------------------------------------



In [19]:

print('---------------------------------------------------------------------------------')
print('Training the model')
print('---------------------------------------------------------------------------------')
print('')

start = time.time()

load_model = True

if load_model == False:
    from tensorflow.keras.models import load_model

    model = load_model("my_dnn_model_hybrid_top20.h5")
    print('Model loaded')

else:

    early_stopping = EarlyStopping(monitor='accuracy', patience=50, restore_best_weights=True)

    # model.fit(X_train, y_train, epochs=1000, batch_size=len(X_train), callbacks=[early_stopping], class_weight=class_weights)
    model.fit(X_train, y_train, epochs=1000, batch_size=len(X_train), callbacks=[early_stopping])


    end = time.time()
    print('---------------------------------------------------------------------------------')
    print('ELAPSE TIME TRAINING MODEL: ',(end - start)/60, 'min')
    print('---------------------------------------------------------------------------------')
    print('')

save_model = True
if save_model == False:
    model.save("my_dnn_model_hybrid_top20.h5")  # Saves model as an HDF5 file


---------------------------------------------------------------------------------
Training the model
---------------------------------------------------------------------------------

Train on 4306071 samples
Epoch 1/1000


2025-02-18 06:42:06.332375: W tensorflow/c/c_api.cc:304] Operation '{name:'training_2/Adam/beta_2/Assign' id:1475 op device:{requested: '', assigned: ''} def:{{{node training_2/Adam/beta_2/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_2/Adam/beta_2, training_2/Adam/beta_2/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


4306071/4306071 [==============================] - 3s 1us/sample - loss: 3.9628 - accuracy: 0.2590
Epoch 2/1000
4306071/4306071 [==============================] - 3s 1us/sample - loss: 2.9967 - accuracy: 0.3324
Epoch 3/1000
4306071/4306071 [==============================] - 3s 1us/sample - loss: 2.2286 - accuracy: 0.3588
Epoch 4/1000
4306071/4306071 [==============================] - 2s 1us/sample - loss: 1.8962 - accuracy: 0.3784
Epoch 5/1000
4306071/4306071 [==============================] - 2s 1us/sample - loss: 1.6727 - accuracy: 0.3950
Epoch 6/1000
4306071/4306071 [==============================] - 3s 1us/sample - loss: 1.5409 - accuracy: 0.4077
Epoch 7/1000
4306071/4306071 [==============================] - 2s 1us/sample - loss: 1.4693 - accuracy: 0.4178
Epoch 8/1000
4306071/4306071 [==============================] - 2s 1us/sample - loss: 1.4265 - accuracy: 0.4271
Epoch 9/1000
4306071/4306071 [==============================] - 3s 1us/sample - loss: 1.3928 - accuracy: 0.4366
Epoch

In [20]:

print('---------------------------------------------------------------------------------')
print('Model Prediction')
print('---------------------------------------------------------------------------------')
print('')
print('---------------------------------------------------------------------------------')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print('ELAPSE TIME MODEL PREDICTION: ',(end - start)/60, 'min')
print('---------------------------------------------------------------------------------')
print('')

#print(y_pred)
ynew = np.argmax(y_pred,axis = 1)
#print(ynew)
score = model.evaluate(X_test, y_test,verbose=1)
#print(score)
pred_label = label[ynew]

#---------------------------------------------------------------------

accuracy =accuracy_score(y_test, ynew)*100
print('accuracy ',accuracy)

label_counts = Counter(y_test)
print(label_counts)

label_counts = Counter(ynew)
print(label_counts)






---------------------------------------------------------------------------------
Model Prediction
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
2025-02-18 08:37:54.220299: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_9/BiasAdd' id:1277 op device:{requested: '', assigned: ''} def:{{{node dense_9/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_9/MatMul, dense_9/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


ELAPSE TIME MODEL PREDICTION:  2.1257039070129395 min
---------------------------------------------------------------------------------



`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
2025-02-18 08:40:01.900989: W tensorflow/c/c_api.cc:304] Operation '{name:'loss_1/mul' id:1341 op device:{requested: '', assigned: ''} def:{{{node loss_1/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_1/mul/x, loss_1/dense_9_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


accuracy  93.27684874061141
Counter({0: 615318, 1: 615084, 2: 615057})
Counter({2: 682660, 1: 622837, 0: 539962})


In [21]:
confusion_metrics('dnn', ynew, y_test)


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
          0         1         2
0  517784.0   18742.0   78792.0
1    7002.0  603908.0    4174.0
2   15176.0     187.0  599694.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9327684874061142
Precision total:  0.9356672488998682
Recall total:  0.9327806808544149
F1 total:  0.9320962034599248
BACC total:  0.9327806808544149
MCC total:  0.9012017778913767


(0.9327684874061142,
 0.9356672488998682,
 0.9327806808544149,
 0.9320962034599248,
 0.9327806808544149,
 0.9012017778913767)

## Integrated Gradients


In [22]:
single_sample_df = X_test.iloc[[0]]

In [23]:
single_sample_df

,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Bwd Packet Length Max,Bwd Packets/s,Packet Length Mean,Avg Bwd Segment Size,Subflow Fwd Bytes,Subflow Bwd Bytes,Init_Win_bytes_backward
5549854,0.000002,0.000018,0.000806,0.444188,0.000003,0.435515,0.5,0.000002,0.000018,0.003494


In [24]:
# %%time
# # Create an analyzer for the model
# analyzer = innvestigate.create_analyzer("integrated_gradients", model)

# analysis = analyzer.analyze(X_test)

# # Perform LRP analysis on a certain number of samples
# # analysis = analyzer.analyze(X_test.sample(10000))

# #uncomment for single sample
# # analysis = analyzer.analyze(single_sample_df)

# names = X_test.columns

# scores = pd.DataFrame(analysis)
# scores_abs = scores.abs()

# # Calculate the sum of each column
# sum_of_columns = scores_abs.sum(axis=0)

# names = list(names)

# sum_of_columns = list(sum_of_columns)

# # Zip the two lists together
# combined = list(zip(names, sum_of_columns))

# # Sort the combined list in descending order based on the values from sum_of_columns
# sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# # Unzip the sorted_combined list to separate names and sum_of_columns
# sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

# print(sorted_names)
# print(sorted_sum_of_columns)

# sorted_names


## LRP


In [25]:
# %%time

# # Create an analyzer for the model
# analyzer = innvestigate.create_analyzer("lrp.z", model)

# # Perform LRP analysis on the input data
# analysis = analyzer.analyze(X_test)

# # Perform LRP analysis on a certain number of samples
# # analysis = analyzer.analyze(X_test.sample(10000))

# #uncomment for single sample
# # analysis = analyzer.analyze(single_sample_df)


# names = X_test.columns
# scores = pd.DataFrame(analysis)
# scores_abs = scores.abs()

# # Calculate the sum of each column
# sum_of_columns = scores_abs.sum(axis=0)

# names = list(names)

# sum_of_columns = list(sum_of_columns)

# # Zip the two lists together
# combined = list(zip(names, sum_of_columns))

# # Sort the combined list in descending order based on the values from sum_of_columns
# sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# # Unzip the sorted_combined list to separate names and sum_of_columns
# sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

# print(sorted_names)
# print(sorted_sum_of_columns)

# sorted_names


## SHAP DEEP Whitebox


In [26]:
%%time

with open(output_file_name, "a") as f:print('',file = f)


print('---------------------------------------------------------------------------------')
print('Generating Explainer')
print('---------------------------------------------------------------------------------')

samples = 1000

# uncomment for single sample
# samples = 1

Label = label

test = X_test
train = X_train
start_index = 0

end_index = samples
explainer = shap.DeepExplainer(model,train[start_index:end_index].values.astype('float'))
shap_values = explainer.shap_values(test[start_index:end_index].values.astype('float'))

vals= np.abs(shap_values).mean(1)
# feature_importance_deep = pd.DataFrame(list(zip(X_train.columns, sum(vals_deep))), columns=['col_name', 'feature_importance_vals_deep'])

feature_importance_deep = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name','feature_importance_vals_deep'])
feature_importance_deep.sort_values(by=['feature_importance_vals_deep'], ascending=False,inplace=True)
feature_importance_deep.head()
print(feature_importance_deep.to_string())



print('---------------------------------------------------------------------------------')
# feature_importance_vals = 'feature_importance_vals'  # Replace with the name of the column you want to extract
feature_val_deep = feature_importance_deep['feature_importance_vals_deep'].tolist()

# col_name = 'col_name'  # Replace with the name of the column you want to extract
feature_name_deep = feature_importance_deep['col_name'].tolist()

# Use zip to combine the two lists, sort based on list1, and then unzip them
zipped_lists = list(zip(feature_name_deep, feature_val_deep))
zipped_lists.sort(key=lambda x: x[1],reverse=True)


with open(output_file_name, "a") as f: print("Whitebox", file=f)
print("Whitebox")


# Convert the sorted result back into separate lists
sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]
# print("SHAP Kernel", file=f)
for k in sorted_list1:
  with open(output_file_name, "a") as f:print("df.pop('",k,"')", sep='',file = f)
  print("df.pop('", k, "'))", sep='')
  
with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
print("Trial_ =[")
for k in sorted_list1:
  with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
  print("'",k,"',", sep='')
with open(output_file_name, "a") as f:print("]", file = f)
print('---------------------------------------------------------------------------------')




---------------------------------------------------------------------------------
Generating Explainer
---------------------------------------------------------------------------------


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


                       col_name  feature_importance_vals_deep
4                 Bwd Packets/s                      0.856827
6          Avg Bwd Segment Size                      0.709590
5            Packet Length Mean                      0.645086
3         Bwd Packet Length Max                      0.438920
9       Init_Win_bytes_backward                      0.157424
2         Fwd Packet Length Max                      0.145773
7             Subflow Fwd Bytes                      0.000848
0   Total Length of Fwd Packets                      0.000846
8             Subflow Bwd Bytes                      0.000804
1   Total Length of Bwd Packets                      0.000797
---------------------------------------------------------------------------------
Whitebox
df.pop(' Bwd Packets/s'))
df.pop(' Avg Bwd Segment Size'))
df.pop(' Packet Length Mean'))
df.pop('Bwd Packet Length Max'))
df.pop(' Init_Win_bytes_backward'))
df.pop(' Fwd Packet Length Max'))
df.pop(' Subflow Fwd Bytes'))
df.p

## SHAP EXPLAINER - BLACKBOX


In [27]:
import shap
import numpy as np
import pandas as pd

print('---------------------------------------------------------------------------------')
print('Generating Explainer')
print('---------------------------------------------------------------------------------')
print('')

# plt.clf()
test = X_test
train = X_train
start_index = 0
end_index = 1000

# Summarize the background data using shap.kmeans
background_data = shap.kmeans(test, 100)  # You can adjust the number of clusters (e.g., 100) as needed

# Use model.predict instead of model.predict_proba
explainer = shap.KernelExplainer(model.predict, background_data)
shap_values = explainer.shap_values(test[start_index:end_index])

# shap.summary_plot(shap_values = shap_values,features = test[start_index:end_index],
# class_names=[label[0],label[1],label[2],label[3],label[4],label[5],label[6]],show=False)
# plt.savefig('KNN_Shap_Summary_global_cicids.png')
# plt.clf()

vals = np.abs(shap_values).mean(1)
feature_importance_kern = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name', 'feature_importance_vals_kernel'])
feature_importance_kern.sort_values(by=['feature_importance_vals_kernel'], ascending=False, inplace=True)
feature_importance_kern.head()
print(feature_importance_kern.to_string())

print('---------------------------------------------------------------------------------')
# feature_importance_vals = 'feature_importance_vals'  # Replace with the name of the column you want to extract
feature_val_kern = feature_importance_kern['feature_importance_vals_kernel'].tolist()

# col_name = 'col_name'  # Replace with the name of the column you want to extract
feature_name_kern = feature_importance_kern['col_name'].tolist()

# for item1, item2 in zip(feature_name, feature_val):
#     print(item1, item2)

# Use zip to combine the two lists, sort based on list1, and then unzip them
zipped_lists = list(zip(feature_name_kern, feature_val_kern))
zipped_lists.sort(key=lambda x: x[1], reverse=True)

# Convert the sorted result back into separate lists
sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]

with open(output_file_name, "a") as f:
    print("whitebox SHAP Kernel", file=f)
print("whitebox SHAP Kernel")

for k in sorted_list1:
    with open(output_file_name, "a") as f:
        print("df.pop('", k, "')", sep='', file=f)
    print("df.pop('", k, "')", sep='')

with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
print("Trial_ =[")
for k in sorted_list1:
  with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
  print("'",k,"',", sep='')
with open(output_file_name, "a") as f:print("]", file = f)
print('---------------------------------------------------------------------------------')

print('---------------------------------------------------------------------------------')

---------------------------------------------------------------------------------
Generating Explainer
---------------------------------------------------------------------------------



100%|██████████| 1000/1000 [49:12<00:00,  2.95s/it]

                       col_name  feature_importance_vals_kernel
5            Packet Length Mean                        5.800278
6          Avg Bwd Segment Size                        4.394660
3         Bwd Packet Length Max                        1.405795
4                 Bwd Packets/s                        1.059787
2         Fwd Packet Length Max                        0.488045
9       Init_Win_bytes_backward                        0.435166
8             Subflow Bwd Bytes                        0.001627
1   Total Length of Bwd Packets                        0.001507
0   Total Length of Fwd Packets                        0.001187
7             Subflow Fwd Bytes                        0.001173
---------------------------------------------------------------------------------
whitebox SHAP Kernel
df.pop(' Packet Length Mean')
df.pop(' Avg Bwd Segment Size')
df.pop('Bwd Packet Length Max')
df.pop(' Bwd Packets/s')
df.pop(' Fwd Packet Length Max')
df.pop(' Init_Win_bytes_backward')
df.pop

### Merge the feature importance dataframes


In [28]:

# Merge the feature importance dataframes
feature_importance_combined = pd.merge(feature_importance_deep, feature_importance_kern, on='col_name')

# Calculate the average feature importance
feature_importance_combined['feature_importance_avg'] = feature_importance_combined[['feature_importance_vals_deep', 'feature_importance_vals_kernel']].mean(axis=1)

# Sort the combined feature importance by the average importance
feature_importance_combined.sort_values(by='feature_importance_avg', ascending=False, inplace=True)

# Extract the sorted feature names
sorted_feature_names = feature_importance_combined['col_name'].tolist()

# Print the combined feature importance
print(feature_importance_combined.to_string())

# Generate df.pop() statements for the sorted feature names

with open(output_file_name, "a") as f:
    print("Merged", file=f)
print("Merged")

for feature in sorted_feature_names:
    with open(output_file_name, "a") as f:
        print("df.pop('", feature, "')", sep='', file=f)
    print("df.pop('", feature, "'))", sep='')

with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
print("Trial_ =[")
for k in sorted_list1:
  with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
  print("'",k,"',", sep='')
with open(output_file_name, "a") as f:print("]", file = f)
print('---------------------------------------------------------------------------------')


                       col_name  feature_importance_vals_deep  feature_importance_vals_kernel  feature_importance_avg
2            Packet Length Mean                      0.645086                        5.800278                3.222682
1          Avg Bwd Segment Size                      0.709590                        4.394660                2.552125
0                 Bwd Packets/s                      0.856827                        1.059787                0.958307
3         Bwd Packet Length Max                      0.438920                        1.405795                0.922357
5         Fwd Packet Length Max                      0.145773                        0.488045                0.316909
4       Init_Win_bytes_backward                      0.157424                        0.435166                0.296295
8             Subflow Bwd Bytes                      0.000804                        0.001627                0.001216
9   Total Length of Bwd Packets                      0.0